In [1]:
from LLM_KG_Human.functions.misc_funcs import *

In [2]:
gen_final()

In [ ]:
os.environ["OPENAI_API_KEY"] = "ENTER-KEY"
openai.api_key = os.getenv("OPENAI_API_KEY")

def token_counter(messages):
    encoding = tiktoken.encoding_for_model("gpt-4")

    total_tokens = 0
    for message in messages:
        tokens = encoding.encode(f"{message['role']}: {message['content']}")
        total_tokens += len(tokens)

    total_tokens += 2 * len(messages)

    return total_tokens

In [ ]:
def back_to_LLM(main_prompt, my_prompt, RBO, co, i):

    issue = """ 
    There were some issue(s) in the action sequence you gave. The error is reflected below. Give me the cot and task sequence again. Make sure to strictly USE All THE INGREDIENTS GIVEN BY THE USER IN THE PROMPT.
    """
    
    # instruction = """ 
    # Additional instructions:
    # - STRICTLY USE THE OBJECTS, TOOLS, RECEPTACLES AND THE ACTIONS GIVEN IN THE ENVIRONMENT. 
    # """

    ll = ['error_occured']

    if os.path.exists("./LLM_KG_Human/output_with_error.txt"):
            
        with open("./LLM_KG_Human/output_with_error.txt", "r") as file:
            lines = file.readlines()

        ll = [l for l in lines if l.strip() != '' and 'Error' in l.split()[0]]

            
    with open (f"./LLM_KG_Human/feedback/LLM_feedback_with_error_{i}.txt" , "w") as file:
        file.write(issue)

    remove_indentation(f"./LLM_KG_Human/feedback/LLM_feedback_with_error_{i}.txt")
    with open (f"./LLM_KG_Human/feedback/LLM_feedback_with_error_{i}.txt" , "a") as file:   
        file.write("\n")

        if not co and RBO=='':
            file.writelines("\nLLM failed to give the action sequence!\n")
        elif RBO == '' and co:
            file.writelines(ll[-1])
        else:
            file.writelines(RBO)

    # with open (f"./LLM_KG_Human/feedback/LLM_feedback_with_error_{i}.txt" , "a") as file:
    #     file.write("\n")
    #     file.write(instruction)


    LLM_feedback_with_error = triple(f"./LLM_KG_Human/feedback/LLM_feedback_with_error_{i}.txt")


    client = OpenAI()

    LLM_orig_output = triple("./LLM_KG_Human/LLM_orig_output.txt")
    LLM_FB = LLM_feedback_with_error
    
    if i==0:
        response = client.chat.completions.create(
                model="gpt-4o", 
                messages=[
                    {"role": "system", "content": main_prompt},
                    {"role": "user", "content": my_prompt },
                    {"role": "assistant", "content": LLM_orig_output},
                    {"role": "user", "content": triple(f"./LLM_KG_Human/feedback/LLM_feedback_with_error_{i}.txt")}
                ],
                stream = True
            )
        
        messages=[
            {"role": "system", "content": main_prompt},
            {"role": "user", "content": my_prompt },
            {"role": "assistant", "content": LLM_orig_output},
            {"role": "user", "content": triple(f"./LLM_KG_Human/feedback/LLM_feedback_with_error_{i}.txt")}
        ]

        token_used = token_counter(messages)

    else :
        response = client.chat.completions.create(
                model="gpt-4o", 
                messages=[
                    {"role": "system", "content": main_prompt},
                    {"role": "user", "content": my_prompt },
                    {"role": "assistant", "content": triple(f"./LLM_KG_Human/feedback/LLM_updated_orig_output_{i}.txt")},
                    {"role": "user", "content": triple(f"./LLM_KG_Human/feedback/LLM_feedback_with_error_{i}.txt")}
                ],
                stream = True
            )
        
        messages=[
            {"role": "system", "content": main_prompt},
            {"role": "user", "content": my_prompt },
            {"role": "assistant", "content": triple(f"./LLM_KG_Human/feedback/LLM_updated_orig_output_{i}.txt")},
            {"role": "user", "content": triple(f"./LLM_KG_Human/feedback/LLM_feedback_with_error_{i}.txt")}
        ]

        token_used = token_counter(messages)

    # elif i==1:
    #     response = client.chat.completions.create(
    #             model=deployment_name, 
    #             messages=[
    #                 {"role": "system", "content": main_prompt},
    #                 {"role": "user", "content": my_prompt },
    #                 {"role": "assistant", "content": LLM_orig_output},
    #                 {"role": "user", "content": triple(f"./feedback/LLM_feedback_with_error_0.txt")},
    #                 {"role": "assistant", "content": triple(f"./feedback/LLM_updated_orig_output_1.txt")},
    #                 {"role": "user", "content": triple(f"./feedback/LLM_feedback_with_error_1.txt")}
    #             ]
    #         )

    # elif i==2:
    #     response = client.chat.completions.create(
    #             model=deployment_name, 
    #             messages=[
    #                 {"role": "system", "content": main_prompt},
    #                 {"role": "user", "content": my_prompt },
    #                 {"role": "assistant", "content": LLM_orig_output},
    #                 {"role": "user", "content": triple(f"./feedback/LLM_feedback_with_error_0.txt")},
    #                 {"role": "assistant", "content": triple(f"./feedback/LLM_updated_orig_output_1.txt")},
    #                 {"role": "user", "content": triple(f"./feedback/LLM_feedback_with_error_1.txt")},
    #                 {"role": "assistant", "content": triple(f"./feedback/LLM_updated_orig_output_2.txt")},
    #                 {"role": "user", "content": triple(f"./feedback/LLM_feedback_with_error_2.txt")}
    #             ]
    #         )


    with open(f"./LLM_KG_Human/feedback/LLM_updated_orig_output_{i+1}.txt", "w") as file:
        for chunk in response:
            if chunk.choices[0].delta.content is not None:
                file.write(chunk.choices[0].delta.content)


    return token_used

In [ ]:
def ask_gpt(main_prompt, my_prompt):

    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o",  
        messages=[
            {"role": "system", "content": main_prompt},
            {"role": "user", "content": my_prompt},
        ],
        stream=True,
        
    )

    messages=[
    {"role": "system", "content": main_prompt},
    {"role": "user", "content": my_prompt },
    ]

    with open("LLM_orig_output.txt", "w") as file:
        for chunk in response:
            if chunk.choices[0].delta.content is not None:
                file.write(chunk.choices[0].delta.content)

    token_used = token_counter(messages)

    return token_used